In [1]:
from torch.nn import MaxPool2d

In [2]:
import torch

In [6]:
x = torch.zeros(1,3,64, 64)

In [4]:
pool_2 = MaxPool2d(2)

In [9]:
pool_2(x).size()

torch.Size([1, 3, 32, 32])

In [11]:
MaxPool2d(4)(x).size()

torch.Size([1, 3, 16, 16])